<h1> Accident Detection From CCTV Footage </h1>

<h2>Description :</h2>
<h3>Dataset Description :</h3>
<p> Accident Detection dataset collected from the CCTV footages containing a total of 990 accident and non-accident frames collected from road videos available on YouTube. The 990 files are split in the 791 training frames, 101 test frames and 98 validation frames.
791 (369-accident, 492-non accident) Training, 101 Test and 98 Validation (52-accident, 46-non accident) frames split in Accident and Non-accident frames in all the three folders. </p>

<h3>Problem Analysis: </h3>
<pre>
Input : Images that can be accident/Non Accident
Output : 0(Indicates No Accident)
         1(Indicates Accident)
</pre>

<h1>1. Loading Data</h1>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
training_data_dir = os.path.join("/kaggle/input/accident-detection-from-cctv-footage/data/train")
training_data = tf.keras.utils.image_dataset_from_directory(
                            training_data_dir,image_size=(256, 256),
                            seed = 42
                            )

In [ ]:
#  iter extracts in each batch of 32 images 
training_data_iter = training_data.as_numpy_iter()
training_batch = training_data_iter.next()

<h1>2. Preprocessing Data </h1>

In [ ]:
# Normalizing rgb pixels value between between 0 & 1 
training_data = training_data.map(lambda x,y: (x/255, y))
training_batch = training_data.as_numpy_iter().next()

# Sanity Check pixel min/max pixel values after normalization
print("Max pixel value : ",training_batch[0].max())
print("Min pixel value : ",training_batch[0].min())

<h2>Loading Validation data for Hyper-parameter Turing</h2>

In [ ]:
validation_data_dir = os.path.join("/kaggle/input/accident-detection-from-cctv-footage/data/val")
validation_data = tf.keras.utils.image_dataset_from_directory(validation_data_dir)
validation_data_iter = validation_data.as_numpy_iter()
validation_batch = validation_data_iter.next()

In [ ]:
# Normalizing Validation data
validation_data = validation_data.map(lambda x,y: (x/255, y))
validation_batch = validation_data.as_numpy_iter().next()

# Sanity Check pixel min/max pixel values after normalization
print("Max pixel value : ",validation_batch[0].max())
print("Min pixel value : ",validation_batch[0].min())

<h1> 3. Building CNN Architecture  </h1>


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Add, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define input layer
inputs = Input(shape=(256, 256, 3))

# First Convolutional Block
x = Conv2D(16, (3,3), 1, activation='relu', padding='same')(inputs)
x = MaxPooling2D()(x)

# Second Convolutional Block with residual connection
conv1 = Conv2D(32, (3,3), 1, activation='relu', padding='same')(x)
conv2 = Conv2D(32, (3,3), 1, activation='relu', padding='same')(conv1)
# Adding convolutional layer to match the number of channels
residual = Conv2D(32, (1, 1), strides=(1, 1), padding='same')(x)
residual = Add()([residual, conv2])
x = MaxPooling2D()(residual)

# Third Convolutional Block with residual connection
conv3 = Conv2D(16, (3,3), 1, activation='relu', padding='same')(x)
conv4 = Conv2D(16, (3,3), 1, activation='relu', padding='same')(conv3)
# Adding convolutional layer to match the number of channels
residual = Conv2D(16, (1, 1), strides=(1, 1), padding='same')(x)
residual = Add()([residual, conv4])
x = MaxPooling2D()(residual)

# Flatten layer
x = Flatten()(x)

# Fully connected layers
x = Dense(256, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

In [ ]:
# model = Sequential()

# model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
# model.add(MaxPooling2D())
# model.add(Conv2D(32, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())
# model.add(Conv2D(16, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())
# model.add(Flatten())
# # Adding neural Layer
# model.add(Dense(256, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [ ]:
learning_rate = 0.001 
optimizer = Adam(learning_rate=learning_rate)
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

<h1> 4.  Training Neural Network </h1>

In [ ]:
# setting up for logging 
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
bst_model = model.fit(training_data, epochs=50, validation_data=validation_data, callbacks=[early_stopping_callback])
bst_model = model.fit(training_data, epochs=50, validation_data=validation_data, callbacks=[tensorboard_callback])
model.save("/kaggle/working/accidents.keras")                                                     

In [ ]:
bst_model.history['validation_accuracy'][-1]

<h2>5.Seeing Training Loss and Accuracy Curve with epochs</h2>

In [ ]:
fig = plt.figure()
plt.plot(bst_model.history['loss'], color='red', label='training loss')
plt.plot(bst_model.history['validation_loss'], color='blue', label='validation_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.xlabel("Epoch")
plt.ylabel("loss")
plt.show()


In [ ]:
fig = plt.figure()
plt.plot(bst_model.history['accuracy'], color='red', label='training accuracy')
plt.plot(bst_model.history['validation_accuracy'], color='blue', label='validation_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

<h1>6. Evaluation</h1>

In [ ]:
test_data_dir = os.path.join("/kaggle/input/accident-detection-from-cctv-footage/data/test")
test_data = tf.keras.utils.image_dataset_from_directory(test_data_dir)
test_data_iter = test_data.as_numpy_iter()
test_batch = test_data_iter.next()

In [ ]:
pre = tf.keras.metrics.Precision
re = tf.keras.metrics.Recall()

In [ ]:
pre = tf.keras.metrics.Precision()
re = tf.keras.metrics.Recall()

for batch in test_data:
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)

In [ ]:
def F1_score(precision, recall):
    return (2*precision*recall)/(precision+recall)

In [ ]:
print("Model achieved an precision score of {:5f}".format(pre.result()))
print("Model achieved an recall score of {:5f}".format(re.result()))

In [ ]:
f1_score = F1_score(pre.result(), re.result())
print("Model achieved an F1-score of {:5f}".format(f1_score))

<h1> 7.Test just to see model working </h1>

In [ ]:
import cv2

# load random samples from samples directory
random_data_dirname = os.path.join("/kaggle/input/accident-detection-from-cctv-footage/data/test/Accident")
pics = [os.path.join(random_data_dirname, filename) for filename in os.listdir(random_data_dirname)]

# load first file from samples
sample = cv2.imread(pics[1], cv2.IMREAD_COLOR)
sample = cv2.resize(sample, (256, 256))

prediction = 1 - model.predict(np.expand_dims(sample/255, 0))

if prediction >= 0.5: 
    label = 'Predicted class is Accident'
else:
    label = 'Predicted class is Not Accident'

plt.title(label)
plt.imshow(sample)
plt.show()

### Create CSV Files for Submission

In [ ]:
import cv2
import pandas as pd

# load random samples from samples directory
test_data_dirname = os.path.join("/kaggle/input/accident-detection-from-cctv-footage/data/test")
pics = [os.path.join(test_data_dirname, filename) for filename in os.listdir(test_data_dirname)]


filenames = []
predictions = []

for dirname in os.listdir(test_data_dirname):
    for filename in os.listdir(os.path.join(test_data_dirname, dirname)):
        if not filename.endswith(".jpg"):
            continue
        filepath = os.path.join(test_data_dirname, dirname, filename)
        
        # load first file from samples
        sample = cv2.imread(filepath, cv2.IMREAD_COLOR)
        sample = cv2.resize(sample, (256, 256))
        
        # predict using model
        prediction = 1 - model.predict(np.expand_dims(sample/255, 0))
        # done because when we loaded data by default 0 label is given to first folder
        # which is Accident but we want just opposite labels
        # we want 0: Accident and 1: Not Accident
        
        filenames.append(filename)
        
        output = 1 if float(prediction[0][0]) >= 0.5 else 0
        predictions.append(output)

df = pd.DataFrame(columns=["ID", "Column ID"])
df["ID"] = filenames
df["Column ID"] = predictions
df.to_csv("/kaggle/working/submission.csv",index=False)